# Prompt Templates

In [3]:
from langchain_openai.llms import OpenAI

llm = OpenAI()

In [5]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
    Responda a seguinte pergunta do usuário: {pergunta}
''')

prompt_template

PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='\n    Responda a seguinte pergunta do usuário: {pergunta}\n')

In [7]:
prompt_template.format(pergunta='O que é um buraco negro?')

'\n    Responda a seguinte pergunta do usuário: O que é um buraco negro?\n'

# Composing prompts | Unindo múltiplos prompts

In [13]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template('''
    Responda a pergunta em até: {n_palavras} palavras
''')

template_lingua = PromptTemplate.from_template('''
    Retorne a respota na {lingua}
''')

template_final = (
    template_word_count
    + template_lingua
    + 'Responda a pergunta seguinte seguindo as intruções: {pergunta}'
)

template_final

PromptTemplate(input_variables=['lingua', 'n_palavras', 'pergunta'], input_types={}, partial_variables={}, template='\n    Responda a pergunta em até: {n_palavras} palavras\n\n    Retorne a respota na {lingua}\nResponda a pergunta seguinte seguindo as intruções: {pergunta}')

In [14]:
prompt = template_final.format(n_palavras=10, lingua='inglês', pergunta='O que é uma estrela?')

In [17]:
print(llm.invoke(prompt))


A star is a luminous celestial body made of gas.


# Templates para Chat

In [18]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template('Essa é minha dúvida: {duvida}')
chat_template.format_messages(duvida='Quem sou eu?')

[HumanMessage(content='Essa é minha dúvida: Quem sou eu?', additional_kwargs={}, response_metadata={})]

In [20]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
        ('human', 'Olá, como vai?'),
        ('ai', 'Melhor agora! Como posso ajuda-lo?'),
        ('human', '{pergunta}')
    ]
)

chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], input_types={}, partial_variables={}, template='Você é um assistente engraçado e se chama {nome_assistente}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Olá, como vai?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Melhor agora! Como posso ajuda-lo?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='{pergunta}'), additional_kwargs={})])

In [21]:
chat_template.format(nome_assistente='Asimo', pergunta='Qual o seu nome?')

'System: Você é um assistente engraçado e se chama Asimo\nHuman: Olá, como vai?\nAI: Melhor agora! Como posso ajuda-lo?\nHuman: Qual o seu nome?'

In [22]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

chat.invoke(chat_template.format(nome_assistente='Asimo', pergunta='Qual o seu nome?'))

AIMessage(content='AI: Meu nome é Asimo, o assistente engraçado! Como posso fazer você rir hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 53, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-83316708-ed70-47ca-aa0e-4a1498792c88-0', usage_metadata={'input_tokens': 53, 'output_tokens': 26, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Templates de Few-shot prompting para llm

In [23]:
exemplos = [
    {"pergunta": "Quem viveu mais tempo, Muhammad Ali ou Alan Turing?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 
""", 
    }, 
    {"pergunta": "Quando nasceu o fundador do craigslist?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 
""", 
    }, 
    {"pergunta": "Quem foi o avô materno de George Washington?",
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi a mãe de George Washington? 
Resposta intermediária: A mãe de George Washington foi Mary Ball Washington. 
Pergunta de acompanhamento: Quem foi o pai de Mary Ball Washington? 
Resposta intermediária: O pai de Mary Ball Washington foi Joseph Ball. 
Então a resposta final é: Joseph Ball 
""", 
    },
    {"pergunta": "Os diretores de Jaws e Casino Royale são do mesmo país?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem é o diretor de Jaws? 
Resposta Intermediária: O diretor de Jaws é Steven Spielberg. 
Pergunta de acompanhamento: De onde é Steven Spielberg? 
Resposta Intermediária: Estados Unidos. 
Pergunta de acompanhamento: Quem é o diretor de Casino Royale? 
Resposta Intermediária: O diretor de Casino Royale é Martin Campbell. 
Pergunta de acompanhamento: De onde é Martin Campbell? 
Resposta Intermediária: Nova Zelândia. 
Então a resposta final é: Não 
""",
    },
]

In [24]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['pergunta', 'resposta'],
    template='Pergunta {pergunta}\n{resposta}'
)

example_prompt.format(**exemplos[0])

'Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'

In [26]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix='Pergunta: {input}',
    input_variables=['input']
)

In [27]:
print(prompt.format(input='Quem fez mais gols, Romário ou Pelé?'))

Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 


Pergunta Quando nasceu o fundador do craigslist?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 


Pergunta Quem foi o avô materno de George Washington?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: 

In [30]:
llm.invoke(prompt.format(input='Quem fez mais gols, Romário ou Pelé?'))

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos gols Romário fez em sua carreira? \nResposta intermediária: Romário fez um total de 772 gols em sua carreira. \nPergunta de acompanhamento: Quantos gols Pelé fez em sua carreira? \nResposta intermediária: Pelé fez um total de 1281 gols em sua carreira. \nEntão a resposta final é: Pelé'

# Templates de Few-shot prompting para chat

In [31]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [35]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{pergunta}'),
    ('ai', '{resposta}')
])

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={})]


In [41]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt = example_prompt
)

prompt_template = ChatPromptTemplate.from_messages([
    few_shot_template,
    ('human', '{input}')
])

prompt = prompt_template.format_messages(input='Quem fez mais gols, Romário ou Pelé?')

In [42]:
chat.invoke(prompt)

AIMessage(content='Pelé fez mais gols em sua carreira do que Romário. Pelé marcou mais de 1200 gols em sua carreira, enquanto Romário marcou cerca de 740 gols. Portanto, Pelé fez mais gols do que Romário.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 538, 'total_tokens': 597, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9fe0d36a-bd18-45a6-bde8-da3eaa2b2790-0', usage_metadata={'input_tokens': 538, 'output_tokens': 59, 'total_tokens': 597, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})